# Manuscript Transmission as Speciation: Using Agent-Based Models and Bayesian Inference

### Digital Approaches to Pre-Modern Texts and Manuscripts (Workshop)

#### Jean-Baptiste Camps, Kelly Christensen, Ulysse Godreau, and Théo Moins

12 June 2025

## Agent based models for manuscript transmission

### Imports

In [ ]:
!git clone https://github.com/LostMa-ERC/simMAtree_workshop.git

import simMAtree_workshop.birth_death_utils as u
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from IPython.core.display import SVG, display
from tqdm.notebook import tqdm
from collections import Counter
import multiprocessing
import os
import pickle

os.chdir('simMAtree_workshop')

### A simple stochastic model of manuscripts transmission: the Birth and Death Model

#### Simulating *arbres réels* and constructing stemmata

Let us first define the function that will simulate a single tree using the constant rate birth and death model

In [ ]:
def generate_tree_bd(lda, mu, Nact, Ninact):
    """
    Generate a tree (arbre réel) according to birth death model.

    Parameters
    ----------
    lda : float
        birth rate of new node per node per iteration
    mu : float
        death rate of nodes per node per per iteration
    Nact : int
        number of iterations of the active reproduction phase
    Ninact : int
        number of iterations of the pure death phase (lda is set to 0)

    Returns
    -------
    G : nx.DiGraph()
        networkx graph object of the generated tree with following node attributes:
            'state' : boolean, True if node living at the end of simulation
            'birth_time' : int
            'death_time' : int

    """
    currentID = 0
    G = nx.DiGraph()
    G.add_node(currentID)
    living = {0:True}

    birth_time = {0:0}
    death_time = {}

    pop = 1
    prob_birth = lda
    prob_death = mu

    for t in range(Nact):
        for current_node in list(G.nodes()):
            r = np.random.rand()
            if r < prob_birth and living[current_node]:
                currentID += 1
                G.add_node(currentID)
                G.add_edge(current_node, currentID)
                living[currentID] = True
                pop += 1
                birth_time[currentID] = t
            if prob_birth < r and r < (prob_birth + prob_death) and living[current_node]:
                living[current_node] =  False
                pop -= 1
                death_time[current_node] = t
        if pop == 0:
            break

    for t in range(Ninact):
        for current_node in list(G.nodes()):
            r = np.random.rand()
            if r <  prob_death and living[current_node]:
                living[current_node] =  False
                pop -= 1
                death_time[current_node] = t + Nact
            if pop == 0:
                break

    nx.set_node_attributes(G, living, 'state')
    nx.set_node_attributes(G, birth_time, 'birth_time')
    nx.set_node_attributes(G, death_time, 'death_time')
    return G

We can now generate a simulated manuscript tradition and play around with parameters, display the corresponding tree of the full tradition as well as the corresponding *stemma codicum*

In [ ]:
λ = 7.9*10**(-3)    # reproduction (=birth) rate
μ = 3.3*10**(-3)    # loss (=death) rate

active_phase_duration = 1000
decimation_phase_duration = 1000

tree = generate_tree_bd(λ, μ, active_phase_duration , decimation_phase_duration)    # generate a simulated full tradition
u.draw_tree(tree, 'arbre_reel')
if any(nx.get_node_attributes(tree,'state').values()):      # check if tradition has any surviving witness
    stemma = u.generate_stemma(tree)
    u.draw_tree(stemma, 'stemma')
    print('=== Full tradition ===')
    display(SVG('arbre_reel.svg'))
    print('=== Stemma ===')
    display(SVG('stemma.svg'))
else:
    print('=== Full tradition ===')
    display(SVG('arbre_reel.svg'))
    print('resulting tradition has no survivng witnesses...Try again !')

#### Simulating manuscript populations over parameter space

We know want to generate whole populations (*i.e.* many trees) for several values of the parameters λ and μ. we define a square region in the parameter space for ```lambda_min <= λ <= lambda_max``` and ```mu_min <= μ <= mu_max``` spanned by ```lambda_mesh * mu_mesh``` evenly spaced points. For each pair ```(λ,μ)```we genrate trad_nb different traditions, *i.e* artificial *texts*.

In [ ]:
lambda_min=6.5*10**(-3)
lambda_max=8.5*10**(-3)
mu_min=3.5*10**(-3)
mu_max=4.5*10**(-3)

lambda_mesh=5
mu_mesh=5

Nact=1000
Ninact=1000

# parameters values
mu_range = np.linspace(mu_max, mu_min, mu_mesh)
lambda_range = np.linspace(lambda_min, lambda_max, lambda_mesh)

# parameter values as displayed in plots
lambda_labels = [r'%.1f'%n for n in (10**(3))*lambda_range]
mu_labels = [r'%.1f'%n for n in (10**(3))*mu_range]

trad_nb=200             # number of generated traditions (=trees)
path='bd_simulations'
output_format='serialized'

The whole set of simulations takes some minutes

In [ ]:
if not os.path.exists(f'{path}'):
    os.mkdir(f'{path}')

progress = tqdm(total = mu_mesh * lambda_mesh * trad_nb)
for i in range(mu_mesh):
    for j in range(lambda_mesh):
        for k in range(trad_nb):
            l = lambda_range[j]
            m = mu_range[i]

            point_path = f'{path}/lambda={lambda_labels[j]}_mu={mu_labels[i]}'

            if not os.path.exists(point_path):
                os.mkdir(point_path)

            g = u.generate_tree_bd(l, m, Nact, Ninact)

            if output_format == 'serialized':
                os.system(f'touch {point_path}/{k}')
                with open(f'{point_path}/{k}', 'wb') as f:
                    pickle.dump(g, f)

            if output_format == 'csv':
                csv_dump(g, f'{point_path}/{k}')

            progress.update(1)

#### Ploting phase diagrams

We may now compute and plot various statistics on simulated data:

- survival rate (proportion of trees with at least one surviving witness)
- median number of witnesses of a tradition
- stemmatic property such as the proportion of bifid stemmata

In [ ]:
def survival_rate(trees):
    n_surv = 0
    for g in trees:
        if any(nx.get_node_attributes(g, 'state').values()):
            n_surv += 1
    return n_surv / len(trees)

def median_witness_number(trees):
    wit_nb = []
    for g in trees:
        wit_nb.append(u.witness_nb(g))
    return np.mean(wit_nb)

def bifidity_rate(trees):
    n_bifid = 0
    n_stemmata = 0
    for g in trees:
        if u.witness_nb(g) >= 3:
            n_stemmata += 1
            st = u.generate_stemma(g)
            rd = st.degree(u.root(st))
            if rd == 2:
                n_bifid += 1
    return n_bifid / n_stemmata

u.plot_phase_diagram(path, lambda_labels, mu_labels, 200, survival_rate, 'survival rate of texts')
u.plot_phase_diagram(path, lambda_labels, mu_labels, 200, median_witness_number, 'median number of witnesses', prec=0)
u.plot_phase_diagram(path, lambda_labels, mu_labels, 200, bifidity_rate, 'proportion of bifid stemmata', prec=2)

#### Abundance data

For some specific values of the parameters λ and μ, we can now look at the distribution of the number of witnesses per texts (*abundance data*)

In [ ]:
λ = 7*10**(-3)
μ = 3*10**(-3)
Ta = 1000
Td = 1000

witness_numbers = []
for k in tqdm(range(1000)):
    g = generate_tree_bd(λ,μ,Ta,Td)
    witness_numbers.append(u.witness_nb(g))

x = Counter(witness_numbers).keys()
y = Counter(witness_numbers).values()
plt.semilogy(x,y,'+')

### A generalization of Birth-Death model: Yule Speciation model

In [ ]:
λ = 7.9*10**(-3)
μ = 3.3*10**(-3)
γ = .5*10**(-3)

active_phase_duration = 1000
decimation_phase_duration = 1000

tree = u.generate_yule_tree(λ,γ,μ, active_phase_duration, decimation_phase_duration)
if any(nx.get_node_attributes(tree,'state').values()):      # check if tradition has any surviving witness
    stemma = u.generate_stemma_yule(tree)
    u.draw_tree_yule(stemma, 'stemma_yule')
    print('=== Multi works stemma ===')
    display(SVG('stemma_yule.svg'))
else:
    print('resulting tradition has no survivng witnesses...Try again !')

#### Abundance data for Yule Model

In [ ]:
witnesses = u.generate_yule_pop(1*10**(-2), 12*10**(-3), 10**(-3), 3.3*10**(-3), 1000, 1000, 2)

In [ ]:
x = Counter(witnesses).keys()
y = Counter(witnesses).values()
plt.loglog(x,y,'+')

## Simulation Based Inference using SimMAtree

#### Package installation

In [ ]:
# Install simmatree directly from GitHub

# !pip install git+https://github.com/LostMa-ERC/simMAtree.git

# Issue on dependancies with colab! Run this 2 lines alternatively:
!pip install git+https://github.com/LostMa-ERC/simMAtree.git --no-deps
!pip install pandas numpy matplotlib seaborn pydantic click rich pyyaml sbi

# Test installation
!simmatree-test

### Imports and Setup

In [ ]:
import os
import tempfile
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import simmatree functions directly
from src.cli.config import Config
from src.cli.generate import generate
from src.cli.inference import inference
from src.cli.score import score

print("All imports successful!")

### Configuration example

In [ ]:
# Define experiment configuration

config_dict = {

    # Type of model we are focusing on
    'model': {
        'name': 'Yule', # 'Yule' or 'BirthDeath' here
        'config': {
            'n_init': 300, # Initial number of trees
            'Nact': 1000, # Number of active iterations
            'Ninact': 1000, # Number of inactive iterations (only deaths)
            'max_pop': 500000 # Maximum population size
        }
    },

    # For generation or scoring : the corresponding parameters (=rate) of a simulation
    'params': {
        'LDA': 0.3,
        'lda': 0.008,
        'gamma': 0.001,
        'mu': 0.0033
    },

    # Configuration of the inference model
    'inference': {
        'name': 'SBI', # For the future : other inference method will be investigated
        'config': {
            'method': 'NPE',
            'num_simulations': 200,
            'num_rounds': 2,
            'random_seed': 42,
            'num_samples': 100,
            'num_workers': 2,       # Reduced for Colab!
            'device': 'cpu'
        }
    }
}

# Create temporary directory for our experiment
temp_dir = "/content/"
config_file = os.path.join(temp_dir, 'Yule_example.yml')

# Save configuration to YAML file
with open(config_file, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# Parse configuration using simmatree's Config class
config = Config(config_file)

print(f"Configuration saved to: {config_file}")


### Abundance Data generation

This simulates the copying and transmission process of manuscripts.

In [ ]:
synthetic_data_file = os.path.join(temp_dir, 'sample_data/synthetic_data.csv')

# Use the generate function directly
success = generate(
    data_path=synthetic_data_file,
    model=config.model,
    parameters=config.params,
    seed=42,
    show_params=False
)

print(f"\nGeneration successful: {success}")
print(f"Synthetic data saved to: {synthetic_data_file}")

# CLI equivalent:
print(f"\n💡 CLI equivalent: simmatree -c {config_file} generate -o {synthetic_data_file} --show-params")

In [ ]:
# Load and examine the synthetic data
df = pd.read_csv(synthetic_data_file, sep=';')

print("\n🔍 First 10 rows:")
print(df.head(10))

# Analyze witness distribution
witness_counts = df.groupby('text_ID')['witness_ID'].count()

print(f"\n📈 Witness Distribution Statistics:")
print(f"Mean number of witnesses per text: {witness_counts.mean():.2f}")
print(f"Median number of witnesses per text: {witness_counts.median():.1f}")
print(f"Max number of witnesses for one text: {witness_counts.max()}")
print(f"Texts with only 1 witness: {(witness_counts == 1).sum()}")

In [ ]:
# Create visualization of witness distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram of witness counts per text
axes[0].hist(witness_counts, bins=40, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Number of Witnesses per Text')
axes[0].set_ylabel('Number of Texts')
axes[0].set_title('Distribution of Witnesses per Text')
axes[0].grid(True, alpha=0.3)

# Log-scale version for better visualization
witness_freq = witness_counts.value_counts().sort_index()
x_values = witness_freq.index.values
y_values = witness_freq.values

axes[1].plot(x_values, y_values, linestyle='--', marker='o',
             markersize=6, linewidth=2, alpha=0.8)
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel('Number of Witnesses per Text (log scale)')
axes[1].set_ylabel('Number of Texts (log scale)')
axes[1].set_title('Distribution of Witnesses per Text (Log-Log Scale)')
axes[1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

##  Comparaison with Birth Death distribution

In [ ]:
# Define another experiment configuration

config_dict["model"]["name"] = "BirthDeath"

config_file_BD = os.path.join(temp_dir, 'BD_example.yml')

# Save configuration to YAML file
with open(config_file_BD, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False)

# Parse configuration using simmatree's Config class
config_BD = Config(config_file)


synthetic_data_BD = os.path.join(temp_dir, 'sample_data/synthetic_data_BD.csv')
success = generate(
    data_path=synthetic_data_BD,
    model=config_BD.model,
    parameters=config_BD.params,
    seed=42,
    show_params=False
)


In [ ]:
# Load and examine the synthetic data
df = pd.read_csv(synthetic_data_BD, sep=';')
witness_counts = df.groupby('text_ID')['witness_ID'].count()

print(f"\n📈 Witness Distribution Statistics:")
print(f"Mean number of witnesses per text: {witness_counts.mean():.2f}")
print(f"Median number of witnesses per text: {witness_counts.median():.1f}")
print(f"Max number of witnesses for one text: {witness_counts.max()}")
print(f"Texts with only 1 witness: {(witness_counts == 1).sum()}")


In [ ]:
# Create visualization of witness distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram of witness counts per text
axes[0].hist(witness_counts, bins=40, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Number of Witnesses per Text')
axes[0].set_ylabel('Number of Texts')
axes[0].set_title('Distribution of Witnesses per Text')
axes[0].grid(True, alpha=0.3)

# Log-scale version for better visualization
witness_freq = witness_counts.value_counts().sort_index()
x_values = witness_freq.index.values
y_values = witness_freq.values

axes[1].plot(x_values, y_values, linestyle='--', marker='o',
             markersize=6, linewidth=2, alpha=0.8)
axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlabel('Number of Witnesses per Text (log scale)')
axes[1].set_ylabel('Number of Texts (log scale)')
axes[1].set_title('Distribution of Witnesses per Text (Log-Log Scale)')
axes[1].grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## Bayesian Inference

This will estimate the model parameters from the observed data.

This may take a few minutes depending on the configuration.

In [ ]:
# Set up results directory
results_dir = Path(temp_dir) / 'inference_results'
results_dir.mkdir(exist_ok=True)

# Run inference using the Python function directly
inference_data = inference(
    csv_file=synthetic_data_file,
    model=config.model,
    backend=config.backend,
    dir=results_dir,
    csv_separator=';'
)

# List generated files
result_files = list(results_dir.glob('*'))
print(f"\n Generated files: {[f.name for f in result_files]}")

# CLI equivalent:
print(f"\n💡 CLI equivalent: simmatree -c {config_file} infer -i {synthetic_data_file} -o {results_dir}")


In [ ]:

# Load posterior summary
posterior_summary = pd.read_csv(results_dir / 'posterior_summary.csv')

print(" Posterior Summary Statistics:")
print(posterior_summary.round(6))

# Compare with true parameters
true_params = config.params
estimated_params = posterior_summary['hpdi_95%'].values

param_names = ['lda', 'mu']
true_values = [true_params[name] for name in param_names]

print(f"\n Parameter Comparison:")
print(f"{'Parameter':<10} {'True Value':<12} {'HPDI Point':<12} {'Relative Error':<15}")
print("-" * 55)

for i, name in enumerate(param_names):
    true_val = true_values[i]
    estimated_val = estimated_params[i]
    rel_error = abs(estimated_val - true_val) / true_val * 100
    print(f"{name:<10} {true_val:<12.6f} {estimated_val:<12.6f} {rel_error:<15.2f}%")

In [ ]:
# Display generated plots
from IPython.display import Image, display
import matplotlib.image as mpimg

# Show posterior distributions
plots_to_show = [
    ('pairplot.png', 'Parameter Correlations and Posterior Distributions'),
    ('posterior.png', 'Marginal Posterior Distributions'),
    ('pp_summaries.png', 'Posterior Predictive Checks')
]

for plot_file, title in plots_to_show:
    plot_path = results_dir / plot_file
    if plot_path.exists():
        print(f"\n{title}")
        img = mpimg.imread(plot_path)
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
        plt.tight_layout()
        plt.show()
    else:
        print(f" Plot not found: {plot_file}")

As we have here the ground truth, we can evaluate how accurate we are in our inference :

In [ ]:
print("Evaluating inference performance against true parameters...")

params_BD = {'lda': config.params['lda'], 'mu': config.params['mu']}

# Run scoring using the Python function directly
score(param_dict=params_BD, results_dir=str(results_dir))

# Load evaluation metrics
metrics_file = results_dir / 'summary_metrics.csv'
if metrics_file.exists():
    metrics = pd.read_csv(metrics_file)
    print("\n Evaluation Metrics:")
    print(metrics.round(6))

    print(f"\n Performance Summary:")
    print(f"Root Mean Square Error (RMSE): {metrics['rmse'].iloc[0]:.6f}")
    print(f"Normalized RMSE: {metrics['nrmse'].iloc[0]:.6f}")
    print(f"Mean Relative Error: {metrics['mean_rel_error_pct'].iloc[0]:.2f}%")
    print(f"Coverage Probability: {metrics['coverage_probability'].iloc[0]:.2f}")
else:
    print(" Evaluation metrics file not found.")

# CLI equivalent:
print(f"\n💡 CLI equivalent: simmatree -c {config_file} score -d {results_dir}")

In [ ]:
# Display generated plots
from IPython.display import Image, display
import matplotlib.image as mpimg

# Show posterior distributions
plots_to_show = [
    ('pairplot.png', 'Parameter Correlations and Posterior Distributions'),
    ('posterior.png', 'Marginal Posterior Distributions'),
    ('relative_error.png', 'Relative error of estimates')
]

for plot_file, title in plots_to_show:
    plot_path = results_dir / plot_file
    if plot_path.exists():
        print(f"\n{title}")
        img = mpimg.imread(plot_path)
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
        plt.tight_layout()
        plt.show()
    else:
        print(f" Plot not found: {plot_file}")